### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
# Laboratory 1 Solution
### Cooperating with: Riccardo Vaccari (s348856)
### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯

## Import

In [16]:
import numpy as np
from tqdm.auto import tqdm

## Setup

In [17]:
NUM_SAMPLES = 10  # number of candidates solution
MAX_STEPS = 10000

In [26]:
# Problem 1 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))


In [30]:
# Problem 2 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [20]:
# Problem 3 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [21]:
# fitness function
def value(sol):
    return VALUES[np.any(sol, axis=0)].sum()

# validity check
def is_valid(sol):
    # each item in at most one knapsack
    if np.any(sol.sum(axis=0) > 1):
        return False
    # each knapsack respects its constraints
    return np.all([WEIGHTS[sol[k]].sum(axis=0) <= CONSTRAINTS[k] for k in range(NUM_KNAPSACKS)])

# tweak: flip one random item in one random knapsack
def tweak(sol):
    new_solution = sol.copy()
    k = rng.integers(0, NUM_KNAPSACKS)
    i = rng.integers(0, NUM_ITEMS)
    new_solution[k, i] = not new_solution[k, i]
    return new_solution

## Base Hill Climbing

In [22]:
def baseHillClimbing(solution):
    for _ in tqdm(range(MAX_STEPS)):
        new_solution = tweak(solution)

        if not is_valid(new_solution):
            continue
        
        # accept if equal or better to avoid remaining stuck
        if value(new_solution) >= value(solution):
            solution = new_solution
    
    return solution

## Hill Climbing (with Tabu Search)

In [23]:
def tabuSearchHillClimbing(solution):
    tabu_list = set()

    for _ in tqdm(range(MAX_STEPS)):
        possible_solutions = [tweak(solution) for _ in range(NUM_SAMPLES)]
        candidates = [
            sol for sol in possible_solutions 
            if is_valid(sol) and sol.flatten().tobytes() not in tabu_list
        ]

        if not candidates:
            continue

        new_solution = max(candidates, key=value)

        # accept if equal or better to avoid remaining stuck
        if value(new_solution) >= value(solution):
            tabu_list.add(solution.flatten().tobytes())
            solution = new_solution
    return solution    


## Results

In [34]:
# start with empty solution (no items assigned)
first_guess = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)

# Results 
solution = tabuSearchHillClimbing(first_guess)
total_value = value(solution)
values_per_knapsack = [VALUES[solution[k]].sum() for k in range(NUM_KNAPSACKS)]

print("Total value:", total_value)
for k, v in enumerate(values_per_knapsack):
    print(f"Knapsack {k+1} value: {v}")

for k in range(NUM_KNAPSACKS):
    total_weights = WEIGHTS[solution[k]].sum(axis=0)
    print(f"Knapsack {k+1} weights: {total_weights}, constraints: {CONSTRAINTS[k]}")

100%|██████████| 10000/10000 [00:01<00:00, 7929.75it/s]

Total value: 40282
Knapsack 1 value: 3374
Knapsack 2 value: 4198
Knapsack 3 value: 3493
Knapsack 4 value: 5188
Knapsack 5 value: 6392
Knapsack 6 value: 3526
Knapsack 7 value: 4968
Knapsack 8 value: 2835
Knapsack 9 value: 2412
Knapsack 10 value: 3896
Knapsack 1 weights: [3032 2624 1650 2897 2328 3292 3640 1312 3099 2114], constraints: [5837 5754 9351 3205 4450 3447 3952 9256 9525 2357]
Knapsack 2 weights: [2882 3322 3497 4240 3148 2572 4163 4022 3879 2858], constraints: [7056 3862 3540 4336 3555 5921 9964 6691 7785 5946]
Knapsack 3 weights: [2043 2336 3644 3216 2527 2224 3207 2464 2027 1449], constraints: [2111 2672 4150 3949 8724 8748 8832 7100 6449 7193]
Knapsack 4 weights: [3045 5714 3503 3001 4544 2413 4411 4396 4445 2918], constraints: [5254 7361 8355 8103 6910 2464 7799 4932 7949 6316]
Knapsack 5 weights: [4500 4602 6413 4600 4328 4351 3002 5041 5610 4500], constraints: [9049 4707 9451 8755 4648 5860 6250 8149 7513 8816]
Knapsack 6 weights: [3871 2952 3588 3001 3416 4364 2948 2415